In [156]:
import pandas as pd
from enum import Enum


class OrderBook:

    class OrderAction(Enum):
        ADD_NEW = ("Add", "Add new")
        REMOVE = ("Remove", "Remove")
        ADD_EXISTING = ("Add", "Add existing")

    class OrderType(Enum):
        BUY = "Buy"
        SELL = "Sell"

    def __init__(self):
        init_open_orders = [
            (1, "Buy", 20.00, 100),
            (2, "Sell", 25.00, 200),
            (4, "Buy", 23.00, 70),
            (5, "Sell", 28.00, 100),
        ]
        self.open_orders = pd.DataFrame(
            init_open_orders, columns=["Order_Id", "Order", "Price", "Quantity"]
        )

        init_order_history = [
            (1, "Buy", "Add", 20.00, 100),
            (2, "Sell", "Add", 25.00, 200),
            (3, "Buy", "Add", 23.00, 50),
            (4, "Buy", "Add", 23.00, 70),
            (3, "Buy", "Remove", 23.00, 50),
            (5, "Sell", "Add", 28.00, 100),
        ]

        self.order_history = pd.DataFrame(
            init_order_history,
            columns=["Order_Id", "Order", "Type", "Price", "Quantity"],
        )

    def display_best_orders(self):
        """Return tuple of dataframes with highest buy and sell prices."""
        df = self.open_orders

        def pick_highest_orders(orders, order_type):
            orders = df[df["Order"] == order_type.value]
            sorted_orders = orders.sort_values(
                by=["Price", "Quantity"], ascending=[False, False]
            )
            highest_price = sorted_orders["Price"].max()
            highest_orders = sorted_orders[sorted_orders["Price"] == highest_price]
            return highest_orders

        highest_buy_orders = pick_highest_orders(df, OrderBook.OrderType.BUY)
        highest_sell_orders = pick_highest_orders(df, OrderBook.OrderType.SELL)

        return highest_buy_orders, highest_sell_orders

    def add_order(self, order_action, **kwargs):
        """Dispatcher method that calls the appropriate add method based on order_action."""
        if order_action == self.OrderAction.ADD_NEW:
            order = self._add_new_order(
                kwargs["order_type"], kwargs["price"], kwargs["quantity"]
            )

        elif order_action == self.OrderAction.ADD_EXISTING:
            order = self._add_existing_order(kwargs["order_id"])

        else:
            raise Exception(
                "Invalid order action. Must be 'ADD_NEW' or 'ADD_EXISTING'."
            )
        self._add_to_history(order, order_action)
        return self.display_best_orders()

    def _add_new_order(self, order_type, price, quantity):
        """Handles adding a new order."""
        if None in (order_type, quantity, price):
            raise Exception(
                "All parameters (order_type, price, quantity) are required."
            )
        next_order_id = self.order_history["Order_Id"].max() + 1
        order = pd.DataFrame(
            [[next_order_id, order_type.value, price, quantity]],
            columns=["Order_Id", "Order", "Price", "Quantity"],
        )
        self.open_orders = pd.concat([self.open_orders, order])
        return order

    def _add_existing_order(self, order_id):
        """Handles adding an existing order."""
        if order_id is None:
            raise Exception("order_id is required.")

        order_from_history = (
            self.order_history[self.order_history["Order_Id"] == order_id]
            .sort_index(ascending=False)
            .head(1)
        )
        if order_from_history.empty:
            raise Exception(f"Order with id {order_id} not found.")
        if order_from_history.iloc[0]["Type"] == self.OrderAction.ADD_EXISTING.value[0]:
            raise Exception(f"Order with id {order_id} has been already added")

        order = order_from_history[["Order_Id", "Order", "Price", "Quantity"]]

        self.open_orders = pd.concat([self.open_orders, order])

        self.display_best_orders()
        return order

    def remove_order(self, order_id):
        """Remove order by id from open orders and add record to history"""
        existing_open_order = self.open_orders[self.open_orders["Order_Id"] == order_id]
        if existing_open_order.empty:
            raise Exception(f"Order with {order_id} not found")
        index_to_drop = self.open_orders[self.open_orders["Order_Id"] == order_id].index
        self.open_orders.drop(index_to_drop, inplace=True)
        self._add_to_history(existing_open_order, OrderBook.OrderAction.REMOVE)
        return self.display_best_orders()

    def _add_to_history(self, order, order_action):
        """Add order to history"""
        order["Type"] = order_action.value[0]
        self.order_history = pd.concat([self.order_history, order], ignore_index=True)
        self.display_best_orders()


orderBook = OrderBook()

print("Open orders")
print(orderBook.open_orders)
print("Order history")
print(orderBook.order_history)

Open orders
   Order_Id Order  Price  Quantity
0         1   Buy   20.0       100
1         2  Sell   25.0       200
2         4   Buy   23.0        70
3         5  Sell   28.0       100
Order history
   Order_Id Order    Type  Price  Quantity
0         1   Buy     Add   20.0       100
1         2  Sell     Add   25.0       200
2         3   Buy     Add   23.0        50
3         4   Buy     Add   23.0        70
4         3   Buy  Remove   23.0        50
5         5  Sell     Add   28.0       100


### Order after adding existing order

In [157]:
print(orderBook.add_order(order_action=orderBook.OrderAction.ADD_EXISTING, order_id=3))

print("Open orders")
print(orderBook.open_orders)
print("Order history")
print(orderBook.order_history)

(   Order_Id Order  Price  Quantity
2         4   Buy   23.0        70
4         3   Buy   23.0        50,    Order_Id Order  Price  Quantity
3         5  Sell   28.0       100)
Open orders
   Order_Id Order  Price  Quantity
0         1   Buy   20.0       100
1         2  Sell   25.0       200
2         4   Buy   23.0        70
3         5  Sell   28.0       100
4         3   Buy   23.0        50
Order history
   Order_Id Order    Type  Price  Quantity
0         1   Buy     Add   20.0       100
1         2  Sell     Add   25.0       200
2         3   Buy     Add   23.0        50
3         4   Buy     Add   23.0        70
4         3   Buy  Remove   23.0        50
5         5  Sell     Add   28.0       100
6         3   Buy     Add   23.0        50


### Order after removing

In [158]:
print(orderBook.remove_order(3))

print("Open orders")
print(orderBook.open_orders)
print("Order history")
print(orderBook.order_history)

(   Order_Id Order  Price  Quantity
2         4   Buy   23.0        70,    Order_Id Order  Price  Quantity
3         5  Sell   28.0       100)
Open orders
   Order_Id Order  Price  Quantity
0         1   Buy   20.0       100
1         2  Sell   25.0       200
2         4   Buy   23.0        70
3         5  Sell   28.0       100
Order history
   Order_Id Order    Type  Price  Quantity
0         1   Buy     Add   20.0       100
1         2  Sell     Add   25.0       200
2         3   Buy     Add   23.0        50
3         4   Buy     Add   23.0        70
4         3   Buy  Remove   23.0        50
5         5  Sell     Add   28.0       100
6         3   Buy     Add   23.0        50
7         3   Buy  Remove   23.0        50


C:\Users\krzyw\AppData\Local\Temp\ipykernel_3520\1883303906.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order["Type"] = order_action.value[0]


### Order after adding new order

In [159]:
print(
    orderBook.add_order(
        order_action=orderBook.OrderAction.ADD_NEW,
        order_type=orderBook.OrderType.BUY,
        price=100,
        quantity=30,
    )
)
print("Open orders")
print(orderBook.open_orders)
print("Order history")
print(orderBook.order_history)

OrderType.BUY 100 30
(   Order_Id Order  Price  Quantity
0         6   Buy  100.0        30,    Order_Id Order  Price  Quantity
3         5  Sell   28.0       100)
Open orders
   Order_Id Order  Price  Quantity
0         1   Buy   20.0       100
1         2  Sell   25.0       200
2         4   Buy   23.0        70
3         5  Sell   28.0       100
0         6   Buy  100.0        30
Order history
   Order_Id Order    Type  Price  Quantity
0         1   Buy     Add   20.0       100
1         2  Sell     Add   25.0       200
2         3   Buy     Add   23.0        50
3         4   Buy     Add   23.0        70
4         3   Buy  Remove   23.0        50
5         5  Sell     Add   28.0       100
6         3   Buy     Add   23.0        50
7         3   Buy  Remove   23.0        50
8         6   Buy     Add  100.0        30
